# Datomic and Datalog Demo

This notebook demonstrates using Datomic with Datalog queries to teach Prolog concepts.

## Running the Example

Open a terminal and run:
```bash
lein run
```

## Prolog Connections

Datalog is inspired by Prolog and shares:
- Logical variables (`?name`)
- Declarative query style
- Rule-based logic
- Unification


In [ ]:
!lein run